In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
import os
import csv
import pandas as pd
from IPython.display import display, clear_output


feedback_file = 'feedback_data.csv'

# 기존 피드백 데이터 로드
if os.path.exists(feedback_file):
    with open(feedback_file, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        feedback_data = list(reader)
else:
    feedback_data = []

def collect_feedback(word, acrostic, user_feedback):
    formatted_acrostic = f'"{acrostic}"'
    feedback_data.append({
        'word': word,
        'acrostic': acrostic,
        'feedback': user_feedback
    })

examples = [
    "'바다' -> '바: 바람이 부는 푸른 바다 다: 다정한 파도가 춤추네.'",
    "'사랑' -> '사: 사랑의 온기를 나누며 랑: 랑데뷰를 꿈꾸는 밤.'",
    "'하늘' -> '하: 하늘은 파랗고 맑아 늘: 늘 우리를 지켜주네.'",
    "'친구' -> '친: 친구는 언제나 곁에 있어 구: 구름처럼 편안한 존재.'",
    "'도서관' -> '도: 도둑이 서: 서울에서 관: 관찰되었다.'",
    "'전공' -> '전: 전공은 나의 꿈 공: 공부는 나의 힘'",
    "'음식' -> '음: 음식점에서 식: 식사를 하자'",
    "'과자' -> '과: 과자는 정말 맛있어 자: 자다가도 입맛을 돋우네'"
]

examples.extend([f"'{feedback_df.loc[i, 'word']}' -> '{feedback_df.loc[i, 'acrostic']}'" for i in positive_feedback.index])
examples.extend([f"'{corrected_feedback_df.loc[i, 'word']}' -> '{corrected_feedback_df.loc[i, 'corrected_acrostic']}'" for i in corrected_feedback_df.index])

example_str = "\n".join(examples)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", f"이 시스템은 사용자가 제공한 단어의 각 글자에 맞는 문구로 창의적인 삼행시(n행시)를 작성해야 합니다. 다음은 예시입니다:\n{example_str} "),
    ("user", "{user_input}"),
])

# 삼행시 생성 후 피드백 수집
while True:
    word = input('단어를 입력해주세요 (종료 => exit) : ')
    if word == 'exit':
        break

    messages = chat_prompt.format_messages(user_input=word)
    llm = ChatOllama(model='llama3.1')
    chain = chat_prompt | llm | StrOutputParser()
    acrostic = chain.invoke({'user_input': word})

    clear_output(wait=True)
    print("\n생성된 삼행시:", acrostic)

    user_feedback = input('이 삼행시에 대한 피드백을 주세요 (good/bad): ')
    collect_feedback(word, acrostic, user_feedback)

# 새로운 피드백 데이터를 기존 데이터와 함께 저장
with open(feedback_file, 'w', newline='', encoding='utf-8') as f:
    fieldnames = ['word', 'acrostic', 'feedback']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(feedback_data)

print("피드백이 저장되었습니다.")


생성된 삼행시: 우: 우주를 감싸는 수많은 별, 주: 주체할 수 없는 아름다움을 간직해.
피드백이 저장되었습니다.


In [19]:
import pandas as pd
import os

# 피드백 데이터 로드
feedback_file = 'feedback_data.csv'
corrected_feedback_file = 'corrected_feedback_data.csv'

feedback_df = pd.read_csv(feedback_file)

# 피드백 데이터 요약
positive_feedback = feedback_df[feedback_df['feedback'] == 'good']
negative_feedback = feedback_df[feedback_df['feedback'] == 'bad']

print("긍정적 피드백 개수:", len(positive_feedback))
print("부정적 피드백 개수:", len(negative_feedback))

# 이미 수정된 부정적 피드백 로드
if os.path.exists(corrected_feedback_file):
    corrected_feedback_df = pd.read_csv(corrected_feedback_file)
else:
    corrected_feedback_df = pd.DataFrame(columns=['word', 'acrostic', 'corrected_acrostic'])

# 부정적인 피드백 데이터를 수정
corrected_examples = []
newly_corrected = False  # 수정된 피드백 여부 확인

for index, row in negative_feedback.iterrows():
    word = row['word']
    acrostic = row['acrostic']
    
    # 이미 수정된 데이터가 있는지 확인
    corrected_row = corrected_feedback_df[(corrected_feedback_df['word'] == word) & (corrected_feedback_df['acrostic'] == acrostic)]
    if not corrected_row.empty and pd.notna(corrected_row.iloc[0]['corrected_acrostic']):
        corrected_acrostic = corrected_row.iloc[0]['corrected_acrostic']
    else:
        print(f"부정적 피드백을 받은 삼행시: {acrostic}")
        corrected_acrostic = input("수정된 삼행시를 입력하세요: ")
        newly_corrected = True
        
        # 새로운 수정 데이터를 추가하거나 업데이트
        if corrected_row.empty:
            new_row = pd.DataFrame({
                'word': [word],
                'acrostic': [acrostic],
                'corrected_acrostic': [corrected_acrostic]
            })
            corrected_feedback_df = pd.concat([corrected_feedback_df, new_row], ignore_index=True)
        else:
            corrected_feedback_df.loc[corrected_row.index, 'corrected_acrostic'] = corrected_acrostic

    corrected_examples.append(corrected_acrostic)

# 새로운 학습 데이터셋 구성
new_examples = positive_feedback['acrostic'].tolist() + corrected_examples

# 수정된 부정적 피드백 데이터 저장
if newly_corrected:  # 새로운 수정이 있을 경우에만 저장
    corrected_feedback_df.to_csv(corrected_feedback_file, index=False)
    print("수정된 부정적 피드백 데이터가 저장되었습니다.")
else:
    print("새로 수정된 피드백이 없습니다.")

긍정적 피드백 개수: 7
부정적 피드백 개수: 13
부정적 피드백을 받은 삼행시: 학교 -> 학: 학생들이 열심히 공부하는 곳, 교: 교실에서 수업이 진행되며, 학교: 학교의 미래를 위해 노력해
수정된 부정적 피드백 데이터가 저장되었습니다.
